## Lignes de niveau, Projet Maths Info, Augustin Jacquillat Louis Perrotin

In [ ]:
Pour ce faire, nous avons séparé en deux grandes étapes le travail à fournir: 
    1. Ecrire une fonction qui réalise le tracé de ces lignes dans un cas simple 